In [1]:
import numpy as np
import os
import zipfile
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import torch

from Mymodule.ModelHandler import *
from Mymodule.Utils import *
from Mymodule.GradCam import *
from Mymodule.BatchHandler import *
from torchvision import models


KeyboardInterrupt



In [ ]:
device = torch.device('cuda:0')
model_name = 'vgg16'

In [ ]:
def explort_imgs(images, parent_dir,candidates):
    for i,image in enumerate(images):
        file_path = os.path.join(parent_dir, f'{candidates[i]}')
        print(file_path)
        cv2.imwrite(file_path, image)
    print('exported..done')
    
def calculate_zero_ratios(img):
    w, h = img.shape
    zeros = 0
    for i in range(w):
        for j in range(w):
            if img[i, j] <= 0:
                zeros += 1
    return zeros / (w*h)

class LayerActivation():
    features = None    
    def __init__(self, model, layer_num):
        self.hook = model.base.features[layer_num].register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.cpu().data.numpy()
    def remove(self):
        self.hook.remove()

        
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


In [ ]:
root = './Dataset/ForGradCam/'
candidates = os.listdir(root)[1:]
candidates

In [ ]:
images = []
for i, candidate in enumerate(candidates):
    if candidate[-3:] != 'png': continue
        
    image_path = os.path.join(root,candidate)
    
    if i <= 4 : 
        x_start = 200
        y_start = 200
        x_end = x_start + 300
        y_end = y_start + 400
    else : 
        x_start = 100
        y_start = 200
        x_end = x_start + 300
        y_end = y_start + 400

    
    image = cv2.imread(image_path)[x_start:x_end, y_start:y_end]
    print(candidate)
    plt.imshow(image)
    plt.show()
    
    image = cv2.resize(image,(140,140))
    images.append(image)
images = np.array(images).copy()
test_y = np.zeros([7,1])
print(images.shape)
print(test_y.shape)

In [ ]:
test_loader = GetLoader([], images, test_y, batch=1, test=True)      

# Show activation map of layers

In [ ]:
def hook_fn(module, input, output):
    activation_maps.append(output.clone())

# ReLU 레이어 후크 등록
def register_hooks(model):
    activation_maps.clear()
    for module in model.modules():
        if isinstance(module, nn.ReLU):
            module.register_forward_hook(hook_fn)

def plot_activation_maps(name,activation_maps):
    fig, axes = plt.subplots(1, len(activation_maps), figsize=(32,32))
    for i, activation_map in enumerate(activation_maps):
        # 채널 평균을 계산하여 3차원으로 축소
        mean_activation_map = activation_map.mean(dim=1, keepdim=True)
        # 이미지 데이터의 형태를 (높이, 너비, 채널)로 변경
        mean_activation_map = mean_activation_map.squeeze().cpu().detach().numpy()
        #.transpose(1, 2, 0)[:,...,1]
    
        # 이미지로 플로팅
        axes[i].imshow(mean_activation_map)
        axes[i].set_title(f'Activation Map {i+1}', size = 15)
        axes[i].axis('off')
    plt.tight_layout()
    plt.savefig(f'./240216_layer_map/{name}')
    plt.show()
    
def plot_activation_maps(name,activation_maps):
    for i, activation_map in enumerate(activation_maps):
        # 채널 평균을 계산하여 3차원으로 축소
        mean_activation_map = activation_map.mean(dim=1, keepdim=True)
        # 이미지 데이터의 형태를 (높이, 너비, 채널)로 변경
        mean_activation_map = mean_activation_map.squeeze().cpu().detach().numpy()
        #.transpose(1, 2, 0)[:,...,1]
    
        # 이미지로 플로팅
        plt.figure(figsize = (7,7), facecolor = 'white')
        plt.imshow(mean_activation_map)
        plt.title(f'Activation Map {i+1}', size = 15)
        plt.tight_layout()
        print(name)
        plt.savefig(f'./240216_layer_map/{name}_{i+1}.png')
        plt.show()

In [ ]:
MRI = 'PD'
train_strategy = 'finetuning'

In [ ]:
# List to store activation maps
for i,(data, label) in enumerate(test_loader):
    activation_maps = []
    data = data.to(device)
    label = label.to(device)

    MRI = candidates[i].split('_')[2][:2]
    model_save_path = f'./Model_for_effusion/vgg16_{MRI}_{train_strategy}.pt'

    model = get_model(model_name, device, pretrained=True)    
    model.load_state_dict(torch.load(model_save_path))
    model.eval()
    model = model.to(device)
    
    register_hooks(model)

    # 순방향 전달
    output = model(data)
    plot_activation_maps(candidates[i],activation_maps)

In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# 압축할 폴더 경로
folder_to_zip = './240216_layer_map/'
# 출력될 압축 파일 경로
output_zip_path = './240216_layer_figures.zip'

# 함수 호출
zip_folder(folder_to_zip, output_zip_path)